In [13]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

In [14]:
data=pd.read_csv("weatherAUS.csv")
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [16]:
cat_col=[]
num_col=[]
for i in data.columns:
    if data[i].dtype=="object":
        cat_col.append(i)
    else:
        num_col.append(i)
print("Kategorik Sutunlar:",cat_col)
print("Numeric Sutunlar:",num_col)

Kategorik Sutunlar: ['Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']
Numeric Sutunlar: ['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm']


In [17]:
data.isnull().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

In [18]:
for i in cat_col:
    data[i].fillna(data[i].mode()[0],inplace=True)

for i in num_col:
    data[i].fillna(data[i].mean(), inplace=True)

In [19]:
data.isnull().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

In [20]:
data["Date"]=pd.to_datetime(data["Date"])
data["Year"]=data["Date"].dt.year
data["Month"]=data["Date"].dt.month
data["Day"]=data["Date"].dt.day
data.drop("Date", axis=1, inplace=True)


data = pd.concat([data,
                   pd.get_dummies(data.Location, drop_first=True),
                   pd.get_dummies(data.WindGustDir, drop_first=True),
                   pd.get_dummies(data.WindDir9am, drop_first=True),
                   pd.get_dummies(data.WindDir3pm, drop_first=True),
                   pd.get_dummies(data.RainToday, drop_first=True),
                   pd.get_dummies(data.RainTomorrow, drop_first=True)], axis=1)

data.drop("Location", axis=1, inplace=True)
data.drop("WindGustDir", axis=1, inplace=True)
data.drop("WindDir9am", axis=1, inplace=True)
data.drop("WindDir3pm", axis=1, inplace=True)


data['RainToday'].replace({"Yes": 1, "No": 0},inplace=True)
data['RainTomorrow'].replace({"Yes": 1, "No": 0},inplace=True)


In [25]:
X=data.drop(["RainTomorrow"],axis=1)
y=data["RainTomorrow"]

X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.1, random_state=52)

In [26]:
logreg=LogisticRegression()
logreg.fit(X_train, y_train)
y_pred_test = logreg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_test)
print('Model Doğruluk Skoru: {0:0.4f}'. format(accuracy_score(y_test, y_pred_test)))

Model Doğruluk Skoru: 0.8593


In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(logreg, X, y, cv=kfold, scoring='accuracy')
print(f"K-Katlı Çapraz Doğrulama Skoru: {scores.mean():.2f} ")